In [94]:
#!pip install csv
#!pip install tweepy
#!pip install nltk
#nltk.download('punkt')
#nltk.download('stopwords')


#!pip install textblob
#!pip install pyspark
#!pip install sagemaker_pyspark

In [2]:
import os
import csv
#import sagemaker_pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField,StringType, FloatType, DateType, LongType, NumericType

from pyspark import SparkContext, SparkConf
#from sagemaker_pyspark import classpath_jars
from pyspark.sql.functions import create_map, struct

In [3]:
import tweepy as tw
import pandas as pd
from pandas import DataFrame
import datetime
import re


import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words("portuguese"))
from nltk.probability import FreqDist


from textblob import TextBlob

In [4]:
#classpath = ":".join(sagemaker_pyspark.classpath_jars())

builder = SparkSession.builder.appName("AnaliseTwitter")
builder.config("spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2")
builder.config("spark.speculation", "false")
builder.config("spark.sql.parquet.compression.codec", "gzip")
builder.config("spark.debug.maxToStringFields", "100")
#builder.config("spark.driver.extraClassPath", classpath)
builder.config("spark.driver.memory", "1g")
builder.config("spark.driver.cores", "1")
builder.config("spark.executor-memory", "20g")
builder.config("spark.executor.cores", "4")

builder.master("local[*]")

spark = builder.getOrCreate()
spark

In [5]:
chave_consumidor = '2gohc6dgizmluhSBDYXoyI2LO'
segredo_consumidor = 'GuJv8qg9rHj8L2ap0gI0pK8SkrkbBDv2n6rIgsbRP1J2HwbVKh'
token_acesso = '290164661-Z9vF5zeEK1lmywCR4HpxNqf7Ofa1ETADL6ktg1Ad'
token_acesso_segredo = '9kkH6xXFjYZre9yPtYnQvf92u0qyhbWxtfrDtGIhFSKtw'

In [6]:
autenticacao = tw.OAuthHandler(chave_consumidor, segredo_consumidor)
autenticacao.set_access_token(token_acesso, token_acesso_segredo)
api = tw.API(autenticacao, wait_on_rate_limit=True)

In [7]:
def remove_url(txt):
    return re.sub(r"http\S+", "", txt)

In [8]:
stop_words.update(('...','!','pra','(',')','O','o','na','de','da','ou',',','http','https',';','%','.','-','@','Vc','a'))

In [82]:
#search_words = "Bolsonaro OR Moro OR Supremo OR Maia"
#search_words = "Sky OR Cancelamento"
search_words = 'copa OR confederação'

date_since = str(datetime.datetime.now()).split(' ')[0]
date_until = str(datetime.datetime.now()+ datetime.timedelta(days=1)).split(' ')[0]

new_search = search_words + " -filter:retweets"

In [83]:
tweets = tw.Cursor(api.search,
                       q=new_search,
                       user_type = 'user',
                       lang='pt',
                       tweet_mode='extended',
                       since=date_since,
                       until=date_until,
                       show_user = True).items(500)

"""SE QUISER BUSCAR OS TWEETs DE UM CONTA ESPECIFICA****************************************
number_of_tweets=200
tweets = api.user_timeline(screen_name='jairbolsonaro',tweet_mode='extended')

lista_tweets = [[tweet.id,\
               tweet.created_at, tweet.user.id, tweet.user.name, \
               tweet.user.screen_name, tweet.user.profile_image_url, \
               #remove_url(str(tweet.full_text)), 
               tweet.user.friends_count, \
               str(tweet.full_text),\
               TextBlob(tweet.full_text).polarity,\
               tweet.user.location] for tweet in tweets]
*****************************************************************************************"""

lista_tweets = [[tweet.id,\
               tweet.created_at, tweet.user.id, tweet.user.name, \
               tweet.user.screen_name, tweet.user.profile_image_url, \
               #remove_url(str(tweet.full_text)), 
               tweet.user.friends_count, \
               str(tweet.full_text),\
               #TextBlob(tweet.full_text).polarity,\
               tweet.user.location] for tweet in tweets] 



In [22]:
def monta_atualiza_datalake(lista):
    
    texto_original = lista[7]
    texto_original = texto_original.encode('latin-1', 'ignore').decode('latin-1')
    texto_original = texto_original.replace('\n','')
    texto_original = texto_original.replace(';','')   
   
    #Remove as referências aos usuários. Exemplo: @SérgioCarlos*******#
    texto_ = re.sub('@[^\s]+','',lista[7])
    texto_ = texto_.replace(';','')  
    #Remove os \n (enter) do texto************************************#
    texto_ = texto_.replace('\n','')
    #Remove as urls do texto******************************************#
    texto_ = remove_url(texto_)
    #Remove os emotions do texto *************************************#
    texto_ = texto_.encode('latin-1', 'ignore').decode('latin-1')

    #Removendo a stopwords **********************************************************#
    tokens_texto_ = nltk.word_tokenize(texto_)
    texto_tweets_=[]
    for w in tokens_texto_:
        if w not in stop_words and len(w) > 3:
            texto_tweets_.append(w.lower())    
        
    linha__tratada =" ".join([texto_tweets_[x] for x in range(len(texto_tweets_))])
    #*********************************************************************************#
    
    #Realiza a analise de sentimento em cima do texto tratado***#
    sentimento = TextBlob(linha__tratada).polarity
    #***********************************************************#
    
    nome_usuario = lista[3].encode('latin-1', 'ignore').decode('latin-1')
    nome_usuario = nome_usuario.replace(';','')
    
    nome_conta = lista[4].encode('latin-1', 'ignore').decode('latin-1')
    nome_conta = nome_conta.replace(';','')
    
    localizacao = lista[8].encode('latin-1', 'ignore').decode('latin-1')
    localizacao = localizacao.replace(';','')
    
    to_append = f'{lista[0]};{lista[1]};{lista[2]};{nome_usuario};{nome_conta};{lista[5]};{lista[6]};{texto_original};{linha__tratada};{sentimento};{localizacao}'

    return to_append    

In [51]:
schema = StructType([StructField('id_tweet', StringType(), True),
                     StructField('data_tweet', StringType(), True),
                     StructField('id_usuario', StringType(), True),
                     StructField('nome_usuario', StringType(), True),
                     StructField('nome_conta', StringType(), True),
                     StructField('foto_usuario', StringType(), True),
                     StructField('qtd_amigos', StringType(), True),
                     StructField('texto_tweet', StringType(), True),
                     StructField('texto_tweet_tratado', StringType(), True),
                     StructField('analise_sentimento', StringType(), True),
                     StructField('localizacao', StringType(), True)
                    ])

In [84]:
pipe_rdd_csv = spark.sparkContext.parallelize(lista_tweets).map(monta_atualiza_datalake)
pipe_rdd_csv = pipe_rdd_csv.map(lambda x : x.split(";"))
rdd = spark.createDataFrame(pipe_rdd_csv,schema)
rdd.show(500, truncate=False)

+-------------------+-------------------+-------------------+--------------------------------------------------+---------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+---------------------------------------------+
|id_tweet           |data_tweet

In [85]:
#Atualiza repositório de twittes************************************************#
path_add = '../repositorio/twittes/'+search_words.lower().replace(' ','_')+'/'
rdd.write.csv(path_add+date_since,sep=';',mode='overwrite')
#*******************************************************************************#


#Adiciona o resgitro de indice do repositório***********************************#
data_hoje = str(datetime.datetime.now()).split(' ')[0]
path_root = os.path.dirname(os.path.realpath('monta_data_lake.ipynb')).replace('analisetweets','')
if (os.path.isfile("../repositorio/indice_repositorios.csv")==False):
    with open("../repositorio/indice_repositorios.csv", "a") as arquivo:
        arq = csv.writer(arquivo, delimiter=';', quotechar=';', quoting=csv.QUOTE_MINIMAL)
        arq.writerows([['tipo_dados','path','filtro','descricao','data_criacao','criado_por','email_crador']])    
        arq.writerows([['Twitter',path_root+'repositorio/twittes/'+search_words.lower().replace(' ','_'), search_words,'Repositorio de dados dos tweets captados diariamente, a partir do filtro de busca definido na coluna filtro.',data_hoje,'Sergio Carlos','scarlos.pe@gmail.com']])
else:
    with open("../repositorio/indice_repositorios.csv", "a") as arquivo:
        arq = csv.writer(arquivo, delimiter=';', quotechar=';', quoting=csv.QUOTE_MINIMAL)
        arq.writerows([['Twitter',path_root+'repositorio/twittes/'+search_words.lower().replace(' ','_'), search_words,'Repositorio de dados dos tweets captados diariamente, a partir do filtro de busca definido na coluna filtro.',data_hoje,'Sergio Carlos','scarlos.pe@gmail.com']])
        print('Adicionou nova linha')
print('Indice atualizado com sucesso.')

Adicionou nova linha
Indice atualizado com sucesso.
